In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

import math
import numpy as np
import requests
import nglview as nv
import ipywidgets as widgets
%matplotlib notebook
import matplotlib.pyplot as plt
from IPython.display import display, display_markdown
from ipywidgets import Layout, HTML
from pathlib import Path

import parmed as pmd
import re

import hublib.use
from hublib.ui import FileUpload, Download
from hublib.cmd import runCommand

from scipy.ndimage import gaussian_filter

%matplotlib notebook
%use gromacs-5.1.4
%use pymol-1.8.4

np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)


HTMLButtonPrompt = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a href="{link}" target="_blank" >
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning" style="width:175px; background-color:#E9E9E9; font-size:10pt; color:black">{text}</button>
</a>
</body>
</html>
'''

HTMLDeadPrompt = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning" style="width:175px; background-color:#E9E9E9; font-size:10pt; color:#D2D2D2">{text}</button>
</body>
</html>
'''

forbidden_strings = ["..", "/", "\\", " ", "~"]

<IPython.core.display.Javascript object>

In [3]:
class md_step:
    def __init__(self, link, linktext, filelist):
        self.link = link
        self.linktext = linktext
        self.linkbox = widgets.HTML(HTMLDeadPrompt.format(text=linktext))
        self.ynbox = widgets.Checkbox(
            value=False,
            description='Validate',
            disabled=True
        )
        self.hbox = widgets.HBox([self.linkbox, self.ynbox])
        self.ynbox.observe(box_on_check)
        self.filelist = filelist
        
    def set_disabled(self, value):
        self.ynbox.disabled = value
        if value==False:
            self.linkbox.value = HTMLButtonPrompt.format(link=self.link, text=self.linktext)
        if value==True:
            self.ynbox.value = False
            self.linkbox.value = HTMLDeadPrompt.format(text=self.linktext)
        
def box_on_check(change):
    global StepList
    Nsteps = len(StepList)
    for n in range(0, Nsteps):
        step = StepList[n]
        if step.ynbox==change['owner']:
            
            # If the user wants to validate the step
            if change['new']==True:
                filecheck = True
                for file in step.filelist:
                    if os.path.isfile("pdb2gmx/"+file)==False:
                        filecheck = False
                        step.ynbox.value = False
                        message_box.value = 'Validation unsuccessful. Essential output files could not be located.'
                        return
            
            if n<Nsteps-1:
                if change['new']==True:
                    nstep = StepList[n+1]
                    nstep.set_disabled(False)
                
                if change['new']==False:
                    for m in range(n+1, Nsteps):
                        nstep = StepList[m]
                        nstep.set_disabled(True)

StepList = [
    md_step('pdb2gmx/pdb2gmx.ipynb', 'Build Protein Topology', ['topol.top', 'protein.gro']),
    md_step('pdb2gmx/add_hydrogens.ipynb', 'Build Pigment Topology', ['pigments.gro']),
    md_step('pdb2gmx/build_complex.ipynb', 'Build Complex', ['complex.top', 'complex.gro']),
]

StepList[0].set_disabled(False)

message_box = widgets.HTML('')

for step in StepList:
    display(step.hbox)
display(message_box)

HTML(value='')